In [1]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver', options=options)

In [3]:
songIds = []
base_url = 'https://www.melon.com/genre/song_list.htm?params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&params%5BdtlGnrCode%5D=params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&gnrCode=GN0'
genre_url = '#params%5BgnrCode%5D=GN0'
index_url = '&startIndex='
for i in tqdm(range(100, 801, 100)):
    main_url = f'{base_url}{i}{genre_url}{i}&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj'
    for k in range(1, 452, 50):
        page_url = f'{main_url}{index_url}{k}'
        driver.get(page_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        trs = soup.select('tbody > tr')
        for tr in trs:
            if tr.select_one('.bullet_icons.age_19'):
                continue
            songId = tr.select_one('.btn.button_icons.type03.song_info')['href'].split("'")[-2]
            songIds.append(songId)

100%|██████████| 8/8 [03:00<00:00, 22.62s/it]


In [4]:
len(songIds)

3970

In [5]:
def nan(data):
    if data != '-':
        return data
    else:
        return np.nan

In [6]:
infos = []
for songId in tqdm(songIds[3600:]):
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
        driver.get(song_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        info = soup.select_one('#conts')
        songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.artist_name').get_text()
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        album = info.select_one('.list > dd').get_text()
        ul = info.select('.list_person.clfix > li')
        lyricist = nan(ul[0].select_one('.artist_name').get_text().strip())
        composer = nan(ul[1].select_one('.artist_name').get_text().strip())
        date = nan(info.select_one('dl > dd:nth-child(4)').get_text().replace('.',''))
        img = info.select_one('.image_typeAll > img')['src']
        comment = info.select_one('#revCnt').get_text().replace('개', '').replace(',', '')
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        lyric = str(info.select_one('#d_video_summary')).split('\n')[-2].replace('\t', '').replace('<br/>', '\n').replace('\n\n', '\n').strip()
        buttons = info.select('.d_djcol_list > ul > li > .thumb > a ')
        plylstSeq = nan(' '.join([button['href'].split(',\'')[-1][:-3] for button in buttons]))
        infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                    'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                    'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})
    except:
        continue

100%|██████████| 370/370 [15:49<00:00,  2.57s/it]


In [7]:
df = pd.DataFrame(infos)
df.shape[0]

360

In [8]:
df.isna().sum()

songId       0
title        0
artist       0
genre        0
album        0
lyricist     0
composer     0
date         0
img          0
comment      0
like         0
lyric        0
plylstSeq    0
dtype: int64

In [9]:
df.to_csv('data/melon_genre13.csv', index=False)

In [10]:
# 모든 csv 파일 합치기
g1 = pd.read_csv('data/melon_genre1.csv')
g2 = pd.read_csv('data/melon_genre2.csv')
g3 = pd.read_csv('data/melon_genre3.csv')
g4 = pd.read_csv('data/melon_genre4.csv')
g5 = pd.read_csv('data/melon_genre5.csv')
g6 = pd.read_csv('data/melon_genre6.csv')
g7 = pd.read_csv('data/melon_genre7.csv')
g8 = pd.read_csv('data/melon_genre8.csv')
g9 = pd.read_csv('data/melon_genre9.csv')
g10 = pd.read_csv('data/melon_genre10.csv')
g11 = pd.read_csv('data/melon_genre11.csv')
g12 = pd.read_csv('data/melon_genre12.csv')
g13 = pd.read_csv('data/melon_genre13.csv')

In [18]:
r1 = pd.concat([g1, g2], axis=0)
r2 = pd.concat([g3, g4], axis=0)
r3 = pd.concat([g5, g6], axis=0)
r4 = pd.concat([g7, g8], axis=0)
r5 = pd.concat([g9, g10], axis=0)
r6 = pd.concat([g11, g12], axis=0)

In [19]:
s1 = pd.concat([r1, r2], axis=0)
s2 = pd.concat([r3, r4], axis=0)
s3 = pd.concat([r5, r6], axis=0)

In [20]:
l1 = pd.concat([s1, s2], axis=0)
l2 = pd.concat([s3, g13], axis=0)

In [21]:
df = pd.concat([l1, l2], axis=0)
df.head(3), df.tail(3)

(     songId         title artist      genre               album lyricist  \
 0  36404853            물론     허각        발라드                  물론      윤사라   
 1  36382580      헤어지자 말해요    박재정        발라드            1집 Alone      박재정   
 2  36394038  Heaven(2023)    임재현  발라드 국내드라마  시작은 첫키스 OST Part.1      강은경   
 
   composer        date                                                img  \
 0      조영수  20230428.0  https://cdnimg.melon.co.kr/cm2/album/images/11...   
 1      박재정  20230420.0  https://cdnimg.melon.co.kr/cm2/album/images/11...   
 2      원상우  20230425.0  https://cdnimg.melon.co.kr/cm2/album/images/11...   
 
    comment   like                                              lyric  \
 0      111  25503  가진 게 그리 많진 않아\n어쩌면 많이 부족할지 몰라\n가끔 나와 다투기도 하겠지만...   
 1      629  34290  헤어지자고 말하려 오늘\n너에게 가다가 우리 추억 생각해 봤어\n처음 본 네 얼굴\...   
 2       66  23345  왜 이제 왔나요 더 야윈 그대\n나만큼 힘들었나요\n두 번 살게 하네요 그대 내 삶...   
 
              plylstSeq  
 0  457624642 457536543  
 1  457536543 405075509  
 2  

In [23]:
df.to_csv('data/melon_genre.csv', index=False)

In [24]:
df = pd.read_csv('data/melon_genre.csv')
df.shape

(3845, 13)

In [25]:
df.songId.nunique()

3510

In [26]:
df2 = pd.read_csv('data/genre/melon_genre_ost_230527.csv')

In [27]:
df2.head(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201,https://cdnimg.melon.co.kr/cm/album/images/000...,332,138215,햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고...,445029956 411111859
1,35609035,인생은 뷰티풀,김호중,발라드 국내영화,인생은 뷰티풀: 비타돌체,김호중,원태연,20220907,https://cdnimg.melon.co.kr/cm2/album/images/11...,197,17317,하나만 알고 둘은\n잘 모르는 사람입니다\n하루는 비틀거리고\n하루는 뒤뚱거리고\n...,457536944 474137187


In [29]:
df2.tail(3)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
944,31980844,나 메셀라,박민성,국내뮤지컬,벤허,왕용범,이성준,20190810,https://cdnimg.melon.co.kr/cm/album/images/103...,3,548,지지리도 가난한 하급 병사의\n아들로 태어났지\n아버진 전쟁에서 어머닌\n전염병으로...,457897578 501077522
945,35062648,The Confrontation,신성록,국내뮤지컬,뮤지컬 '지킬앤하이드 (Jekyll & Hyde)' 2021 Korean Cast ...,Leslie Bricusse,Frank Wildhorn,20220511,https://cdnimg.melon.co.kr/cm2/album/images/10...,1,182,밤 검은 어둠\n길 잃은 영혼\n새벽은 멀고\n끝없는 밤\n세상 그 무엇도\n날 막...,471800202
946,2582556,마음 굳게 먹어라,서범석,국내뮤지컬,뮤지컬 모차르트! OST,Michael Kunze,Sylvester Levay,20100218,https://cdnimg.melon.co.kr/cm/album/images/008...,1,344,왜 허락했지 \n왜 허락했나 \n널 떠나 보낸 내가 싫구나 \n불안한 생각들 \n경...,457897578 423636590


In [28]:
df3 = pd.concat([df, df2], axis=0)
df3.head(3), df3.tail(3)

(     songId         title artist      genre               album lyricist  \
 0  36404853            물론     허각        발라드                  물론      윤사라   
 1  36382580      헤어지자 말해요    박재정        발라드            1집 Alone      박재정   
 2  36394038  Heaven(2023)    임재현  발라드 국내드라마  시작은 첫키스 OST Part.1      강은경   
 
   composer        date                                                img  \
 0      조영수  20230428.0  https://cdnimg.melon.co.kr/cm2/album/images/11...   
 1      박재정  20230420.0  https://cdnimg.melon.co.kr/cm2/album/images/11...   
 2      원상우  20230425.0  https://cdnimg.melon.co.kr/cm2/album/images/11...   
 
   comment   like                                              lyric  \
 0     111  25503  가진 게 그리 많진 않아\n어쩌면 많이 부족할지 몰라\n가끔 나와 다투기도 하겠지만...   
 1     629  34290  헤어지자고 말하려 오늘\n너에게 가다가 우리 추억 생각해 봤어\n처음 본 네 얼굴\...   
 2      66  23345  왜 이제 왔나요 더 야윈 그대\n나만큼 힘들었나요\n두 번 살게 하네요 그대 내 삶...   
 
              plylstSeq  
 0  457624642 457536543  
 1  457536543 405075509  
 2  4575

In [31]:
df3.set_index('songId', inplace=True)
df3.reset_index(inplace=True)

In [33]:
df3.to_csv('data/genre/melon_genre_concat.csv')